# **Distribución % de créditos por segmento (trimestre)**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "pachuca" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_pachuca',
 'df_financiamientos_2020_pachuca',
 'df_financiamientos_2021_pachuca',
 'df_financiamientos_2022_pachuca',
 'df_financiamientos_2023_pachuca',
 'df_financiamientos_2024_pachuca']

In [2]:
df_financiamientos_2022_pachuca= df_financiamientos_2022_pachuca[['id','mes','monto']]
df_financiamientos_2023_pachuca= df_financiamientos_2023_pachuca[['id','mes','monto']]
df_financiamientos_2024_pachuca= df_financiamientos_2024_pachuca[['id','mes','monto']]

## 2022

In [3]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2022_pachuca['trimestre'] = df_financiamientos_2022_pachuca['mes'].apply(asignar_trimestre)
df_financiamientos_2022_pachuca

,id,mes,monto,trimestre
0,1,1,0.00,1
1,2,1,1563551.33,1
2,3,1,1263770.32,1
3,4,1,3614324.42,1
4,5,1,252907.18,1
...,...,...,...,...
1383,1384,12,0.00,4
1384,1385,12,0.00,4
1385,1386,12,0.00,4
1386,1387,12,0.00,4


In [4]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2022_pachuca['segmento'] = df_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2022_pachuca.head()

,id,mes,monto,trimestre,segmento
0,1,1,0.00,1,E
1,2,1,1563551.33,1,D
2,3,1,1263770.32,1,D
3,4,1,3614324.42,1,A
4,5,1,252907.18,1,E


In [5]:
trimestres_credit_2022_pachuca = df_financiamientos_2022_pachuca[['trimestre','segmento']]
trimestres_credit_2022_pachuca

,trimestre,segmento
0,1,E
1,1,D
2,1,D
3,1,A
4,1,E
...,...,...
1383,4,E
1384,4,E
1385,4,E
1386,4,E


In [6]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2022_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2022_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  1388
trimestre          1          2          3          4
segmento                                             
A           0.576369   0.504323   0.720461   0.720461
B           1.080692   1.512968   1.080692   1.585014
C           2.953890   2.521614   2.233429   2.593660
D           4.610951   5.043228   5.475504   4.682997
E          13.112392  12.103746  12.752161  16.642651
L           0.216138   0.144092   0.288184   0.432277
S           1.152738   1.440922   2.017291   1.801153


In [7]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2022_pachuca)
print("Total de registros: ", total_registros)

# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2022_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

# Definir colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Crear la figura de la gráfica
fig = go.Figure()
trimestres = tabla_porcentajes.columns

# Reordenar la tabla de porcentajes según el orden de segmentos
tabla_porcentajes = tabla_porcentajes.reindex(orden_segmentos)

# Agregar trazos para cada trimestre
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))

# Actualizar diseño de la gráfica
fig.update_layout(
    yaxis=dict(
        title='%',  # Título del eje y
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y         
    ),    
    xaxis=dict(
        title='Segmentos',  # Título del eje x
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title="Trimestre",
    barmode='group',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
     # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_percent_2022', carpeta='assets/graficas')

# Mostrar la gráfica
fig.show()


Total de registros:  1388
trimestre          1          2          3          4
segmento                                             
A           0.576369   0.504323   0.720461   0.720461
B           1.080692   1.512968   1.080692   1.585014
C           2.953890   2.521614   2.233429   2.593660
D           4.610951   5.043228   5.475504   4.682997
E          13.112392  12.103746  12.752161  16.642651
L           0.216138   0.144092   0.288184   0.432277
S           1.152738   1.440922   2.017291   1.801153


## 2023

In [8]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2023_pachuca['trimestre'] = df_financiamientos_2023_pachuca['mes'].apply(asignar_trimestre)
df_financiamientos_2023_pachuca

,id,mes,monto,trimestre
0,1,1,0.00,1
1,2,1,486903.48,1
2,3,1,5702163.94,1
3,4,1,5284544.47,1
4,5,1,691154.29,1
...,...,...,...,...
1394,1395,12,1740000.00,4
1395,1396,12,395789.35,4
1396,1397,12,1451000.00,4
1397,1398,12,0.00,4


In [9]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2023_pachuca['segmento'] = df_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2023_pachuca.head()

,id,mes,monto,trimestre,segmento
0,1,1,0.00,1,E
1,2,1,486903.48,1,E
2,3,1,5702163.94,1,S
3,4,1,5284544.47,1,S
4,5,1,691154.29,1,E


In [10]:
trimestres_credit_2023_pachuca = df_financiamientos_2023_pachuca[['trimestre','segmento']]
trimestres_credit_2023_pachuca

,trimestre,segmento
0,1,E
1,1,E
2,1,S
3,1,S
4,1,E
...,...,...
1394,4,D
1395,4,E
1396,4,D
1397,4,E


In [11]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2023_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2023_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  1399
trimestre          1          2          3          4
segmento                                             
A           0.714796   0.571837   0.857756   0.643317
B           1.072194   0.929235   1.215154   1.715511
C           2.287348   1.786991   1.644031   1.786991
D           3.716941   6.433167   5.360972   5.861330
E          12.866333  16.440315  10.864904  16.297355
ELITE       0.000000   0.000000   0.142959   0.000000
L           0.000000   0.142959   0.142959   0.142959
S           1.286633   1.429593   1.858470   1.786991


In [12]:


# Definir colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Crear la figura de la gráfica
fig = go.Figure()
trimestres = tabla_porcentajes.columns
# Reordenar la tabla de porcentajes según el orden de segmentos
tabla_porcentajes = tabla_porcentajes.reindex(orden_segmentos)
# Agregar trazos para cada trimestre
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))

# Actualizar diseño de la gráfica
fig.update_layout(
    yaxis=dict(
        title='%',  # Título del eje y
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y         
    ),    
    xaxis=dict(
        title='Segmentos',  # Título del eje x
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title="Trimestre",
    barmode='group',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_percent_2023', carpeta='assets/graficas')

# Mostrar la gráfica
fig.show()


## 2024

In [13]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2024_pachuca['trimestre'] = df_financiamientos_2024_pachuca['mes'].apply(asignar_trimestre)
df_financiamientos_2024_pachuca

,id,mes,monto,trimestre
0,1,1,5479808.78,1
1,2,1,1715027.31,1
2,3,1,938335.09,1
3,4,1,1014831.18,1
4,5,1,2344630.70,1
...,...,...,...,...
787,788,7,1596000.00,3
788,789,7,1978676.33,3
789,790,7,0.00,3
790,791,7,0.00,3


In [14]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2024_pachuca['segmento'] = df_financiamientos_2024_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2024_pachuca.head()

,id,mes,monto,trimestre,segmento
0,1,1,5479808.78,1,S
1,2,1,1715027.31,1,D
2,3,1,938335.09,1,E
3,4,1,1014831.18,1,D
4,5,1,2344630.70,1,C


In [15]:
trimestres_credit_2023_pachuca = df_financiamientos_2024_pachuca[['trimestre','segmento']]
trimestres_credit_2023_pachuca

,trimestre,segmento
0,1,S
1,1,D
2,1,E
3,1,D
4,1,C
...,...,...
787,3,D
788,3,C
789,3,E
790,3,E


In [16]:
trimestres_credit_2023_pachuca = trimestres_credit_2023_pachuca[trimestres_credit_2023_pachuca['trimestre']!=3]

In [17]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2023_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2023_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  651
trimestre          1          2
segmento                       
A           1.843318   2.150538
B           3.072197   2.611367
C           3.379416   5.069124
D           9.062980  10.906298
E          28.110599  26.574501
L           0.000000   0.153610
S           2.918587   4.147465


In [18]:


# Definir colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
orden_segmentos=["E", "D", "C", "B","A", "S", "L", 'ELITE']

# Crear la figura de la gráfica
fig = go.Figure()
trimestres = tabla_porcentajes.columns
# Reordenar la tabla de porcentajes según el orden de segmentos
tabla_porcentajes = tabla_porcentajes.reindex(orden_segmentos)

# Agregar trazos para cada trimestre
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))

# Actualizar diseño de la gráfica
fig.update_layout(
    yaxis=dict(
        title='%',  # Título del eje y
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y         
    ),    
    xaxis=dict(
        title='Segmentos',  # Título del eje x
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title="Trimestre",
    barmode='group',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
     # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_percent_2024', carpeta='assets/graficas')

# Mostrar la gráfica
fig.show()
